**本节介绍torch框架的最大特色：automatic differentiation，即自动微分求导机制**

In [33]:
# 导入包
import torch

In [34]:
x = torch.arange(4.0,dtype = torch.float32) # 仅浮点数有requires_grad
x.requires_grad = True
print(x.requires_grad)
print(x.grad) # 默认为None

# 上面等价于x = torch.arange(0,11,1,dtype = torch.float32,requires_grad = True)

True
None


In [35]:
print(x.shape)
print(x)
print(torch.dot(x,x)) # torch的点积会自动对齐

torch.Size([4])
tensor([0., 1., 2., 3.], requires_grad=True)
tensor(14., grad_fn=<DotBackward0>)


In [36]:
y = 2 * torch.dot(x,x)
print(y)

tensor(28., grad_fn=<MulBackward0>)


In [37]:
y.backward()
print(x.grad)
print(4 * x == x.grad) # 理论与实际相等

tensor([ 0.,  4.,  8., 12.])
tensor([True, True, True, True])


In [38]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_() # 清空梯度
y = x.sum()
y.backward()
print(x.grad)

tensor([1., 1., 1., 1.])


**注意：上面的实际都是标量对x的求导，即一个值函数，如果结果是张量呢？**

解决方案：标量可以直接backword()，（我的理解）而张量在实际中最终必然通过运算转化到标量，故可以知道张量在计算图中通常是中间态，必然有上游梯度，所以在对张量进行backward()时，要传入对应当前张量shape的梯度.

In [40]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
print(x.grad)

tensor([0., 2., 4., 6.])


有时，我们希望将某些计算移动到记录的计算图之外。例如，假设y是作为x的函数计算的，而z则是作为y和x的
函数计算的。想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数，并且只考虑
到x在y被计算后发挥的作用。
这里可以分离y来返回一个新变量u，该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息。换句
话说，梯度不会向后流经u到x。因此，下面的反向传播函数计算$z=u*x$关于x的偏导数，同时将u作为常数处理，
而不是$z=x*x*x$关于x的偏导数。

In [42]:
# 分离计算
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
print(x.grad == u)
# 即把y当成常量

tensor([True, True, True, True])


In [43]:
# 由于记录了y的计算结果，我们可以随后在y上调用反向传播，得到y=x*x关于的x的导数，即2*x
x.grad.zero_()
y.sum().backward()
print(x.grad == 2 * x)

tensor([True, True, True, True])
